In [239]:
!pip3 install sympy
from sympy import Symbol, diff, hessian
import numpy as np

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [240]:
n_variaveis = 2

x_1 = Symbol('x_1')
x_2 = Symbol('x_2')

funcao = (x_1**2 + x_2 - 11)**2 + (x_1 + x_2**2 - 7)**2
funcao

(x_1 + x_2**2 - 7)**2 + (x_1**2 + x_2 - 11)**2

In [241]:
def calcula_gradiente(funcao, variaveis):
    return [diff(funcao, variavel) for variavel in variaveis]

def calcula_hessiana(funcao, variaveis):
  return hessian(funcao, variaveis)

In [242]:
gradiente = calcula_gradiente(funcao, [x_1, x_2])
gradiente

[4*x_1*(x_1**2 + x_2 - 11) + 2*x_1 + 2*x_2**2 - 14,
 2*x_1**2 + 4*x_2*(x_1 + x_2**2 - 7) + 2*x_2 - 22]

In [243]:
hessiana = calcula_hessiana(funcao, [x_1, x_2])
hessiana

Matrix([
[12*x_1**2 + 4*x_2 - 42,          4*x_1 + 4*x_2],
[         4*x_1 + 4*x_2, 4*x_1 + 12*x_2**2 - 26]])

In [244]:
def substitui_variaveis_funcao(funcao, variaveis, ponto):
    return funcao.subs(dict(zip(variaveis, ponto)))

def substitui_variaveis_gradiente(gradiente, variaveis, ponto):
    return np.array([gradiente_i.subs(dict(zip(variaveis, ponto))) for gradiente_i in gradiente]).astype(np.float64)

def substitui_variaveis_hessiana(hessiana, variaveis, ponto):
    return np.array(hessiana.subs(dict(zip(variaveis, ponto)))).astype(np.float64)

In [245]:
def calcula_bfgs(H, p, q):
    termo_2_1 = (1 + (((q.T @ H) @ q) / (p.T @ q)))
    termo_2_2 = (p @ p.T) / (p.T @ q)
    termo_2 = termo_2_1 * termo_2_2
    termo_3 = ((p @ (q.T @ H)) + ((H @ q) @ p.T)) / (p.T @ q)
    return H + termo_2 - termo_3

In [246]:
def calcula_dfp(H, p, q):
    termo_2 = (p.T @ p) / (p.T @ q)
    termo_3 = ((H @ q) @ (q.T @ H)) / ((q.T @ H) @ q)
    return H + termo_2 - termo_3

![Armijo's Rule](./img/alg_busca_armijo.png)

In [247]:
def busca_de_armijo(funcao: any, gradiente: any, variaveis: list, ponto_inicial: np.array, d: np.array, gama: float, eta: float):
  """
  Parametros:
    - funcao: função a ser minimizada
    - gradiente: gradiente da função a ser minimizada
    - variaveis: lista de variáveis da função
    - ponto_inicial: ponto inicial
    - d: direção de descida
    - gama: fator de redução do passo
    - eta: fator de ???

  Retorna:
    - t: passo ótimo
    - iteracoes: número de iterações
  """
  t = 1
  iteracoes = 1
  while substitui_variaveis_funcao(funcao, variaveis, ponto_inicial + t*d) > substitui_variaveis_funcao(funcao, variaveis, ponto_inicial) + eta*t*np.dot(substitui_variaveis_gradiente(gradiente, variaveis, ponto_inicial), d):
    t = gama*t
    iteracoes += 1

  return t, iteracoes

<h2>Método do Gradiente</h2>
<img src="./img/alg_metodo_gradiente.png" width="500">


In [248]:
def metodo_do_gradiente(
    funcao: any,
    gradiente: any,
    variaveis: list,
    ponto_inicial: np.array,
    gama: float,
    eta: float,
    maximo_iteracoes: int = 1000,
    valor_minimo: float = 1e-20):

  """
  Parametros:
    - funcao: função a ser minimizada
    - gradiente: gradiente da função a ser minimizada
    - variaveis: lista de variáveis da função
    - ponto_inicial: ponto inicial
    - gama: fator de redução do passo
    - eta: fator de ???
    - maximo_iteracoes: número máximo de iterações
    - valor_minimo: valor mínimo da função para parar a execução

  Retorna:
    - ponto: ponto ótimo
    - iteracoes: número de iterações
    - iteracoes_armijo: número de iterações da busca de armijo
  """

  iteracoes = 0
  iteracoes_armijo = 0
  ponto = ponto_inicial
  while np.linalg.norm(substitui_variaveis_gradiente(gradiente, variaveis, ponto)) > valor_minimo:
    d = -substitui_variaveis_gradiente(gradiente, variaveis, ponto)
    t, iteracoes_armijo_tmp = busca_de_armijo(funcao, gradiente, variaveis, ponto, d, gama, eta)
    iteracoes_armijo += iteracoes_armijo_tmp
    ponto = ponto + t*d
    if iteracoes > maximo_iteracoes:
      print('Número máximo de iterações atingido')
      break
    iteracoes += 1

  return ponto, iteracoes, iteracoes_armijo

<h2>Método de Newton</h2>
<img src="./img/alg_metodo_newton.png" width="500">

In [249]:
def metodo_de_newton(
    funcao: any,
    gradiente: any,
    hessiana: any,
    variaveis: list,
    ponto_inicial: np.array,
    gama: float,
    eta: float,
    maximo_iteracoes: int = 1000,
    valor_minimo: float = 1e-20):

  """
  Parametros:
    - funcao: função a ser minimizada
    - gradiente: gradiente da função a ser minimizada
    - hessiana: hessiana da função a ser minimizada
    - variaveis: lista de variáveis da função
    - ponto_inicial: ponto inicial
    - gama: fator de redução do passo
    - eta: fator de ???
    - maximo_iteracoes: número máximo de iterações
    - valor_minimo: valor mínimo da função para parar a execução

  Retorna:
    - ponto: ponto ótimo
    - iteracoes: número de iterações
    - iteracoes_armijo: número de iterações da busca de armijo
  """

  iteracoes = 0
  iteracoes_armijo = 0
  ponto = ponto_inicial

  while np.linalg.norm(substitui_variaveis_gradiente(gradiente, variaveis, ponto)) > valor_minimo:
    d = -np.linalg.inv(substitui_variaveis_hessiana(hessiana, variaveis, ponto)) @ substitui_variaveis_gradiente(gradiente, variaveis, ponto)
    t, iteracoes_armijo_tmp = busca_de_armijo(funcao, gradiente, variaveis, ponto, d, gama, eta)
    iteracoes_armijo += iteracoes_armijo_tmp
    ponto = ponto + t*d
    if iteracoes > maximo_iteracoes:
      print('Número máximo de iterações atingido')
      break
    iteracoes += 1

  return ponto, iteracoes, iteracoes_armijo

<h2>Método de Quase-Newton</h2>
<img src="./img/alg_metodo_quase_newton.png" width="500">

In [250]:
def metodo_de_quase_newton(
    funcao: any,
    n_variaveis: int,
    gradiente: any,
    variaveis: list,
    ponto_inicial: np.array,
    gama: float,
    eta: float,
    maximo_iteracoes: int = 1000,
    valor_minimo: float = 1e-20,
    metodo: str = 'dfp'
    ):

  """
  Parametros:
    - funcao: função a ser minimizada
    - gradiente: gradiente da função a ser minimizada
    - hessiana: hessiana da função a ser minimizada
    - variaveis: lista de variáveis da função
    - ponto_inicial: ponto inicial
    - gama: fator de redução do passo
    - eta: fator de ???
    - maximo_iteracoes: número máximo de iterações
    - valor_minimo: valor mínimo da função para parar a execução
    - metodo: método de atualização

  Retorna:
    - ponto: ponto ótimo
    - iteracoes: número de iterações
    - iteracoes_armijo: número de iterações da busca de armijo
  """

  iteracoes = 0
  iteracoes_armijo = 0
  ponto = ponto_inicial
  H = np.eye(n_variaveis)

  while np.linalg.norm(substitui_variaveis_gradiente(gradiente, variaveis, ponto)) > valor_minimo:
    d = -H @ substitui_variaveis_gradiente(gradiente, variaveis, ponto)
    print(d)
    t, iteracoes_armijo_tmp = busca_de_armijo(funcao, gradiente, variaveis, ponto, d, gama, eta)
    iteracoes_armijo += iteracoes_armijo_tmp

    proximo_ponto = ponto + t*d
    p = proximo_ponto - ponto
    q = substitui_variaveis_gradiente(gradiente, variaveis, proximo_ponto) - substitui_variaveis_gradiente(gradiente, variaveis, ponto)
    ponto = proximo_ponto

    if iteracoes > maximo_iteracoes:
      print('Número máximo de iterações atingido')
      break
    iteracoes += 1
    H = calcula_dfp(H, p, q) if metodo == 'dfp' else calcula_bfgs(H, p, q)

  return ponto, iteracoes, iteracoes_armijo

In [251]:
from sympy import symbols

# Define the variables x1 and x2
x1, x2 = symbols('x1 x2')

# Define the function
f_x = (1/2) * (x1 - 2)**2 + (x2 - 1)**2
f_x
print(f_x)
# Define the gradient
grad_f_x = [f_x.diff(x1), f_x.diff(x2)]
print(grad_f_x)



0.5*(x1 - 2)**2 + (x2 - 1)**2
[1.0*x1 - 2.0, 2*x2 - 2]


In [252]:
from sympy import symbols

# Define the variables x1 and x2
variaveis = list(symbols('x1 x2'))
print(variaveis)
x1, x2 = variaveis

# Define the function
f_x = (1/2) * (x1 - 2)**2 + (x2 - 1)**2
f_x
print(f_x)

# Define the gradient
grad_f_x = calcula_gradiente(f_x, variaveis)
print(grad_f_x)

# Define the hessian
hess_f_x = calcula_hessiana(f_x, variaveis)
print(hess_f_x)


[x1, x2]
0.5*(x1 - 2)**2 + (x2 - 1)**2
[1.0*x1 - 2.0, 2*x2 - 2]
Matrix([[1.00000000000000, 0], [0, 2]])


In [271]:
ponto_inicial = np.array([2.01, 1.01])
d = np.array([3, 1])
eta = 0.02
gama = 0.08
hessiana = calcula_hessiana(f_x, variaveis)

passo_otimo, n_iteracoes = busca_de_armijo(f_x, grad_f_x, variaveis, ponto_inicial, d, gama, eta)
passo_otimo, n_iteracoes

(3.518437208883201e-17, 16)

In [272]:
metodo_do_gradiente(f_x, grad_f_x, variaveis, ponto_inicial, gama, eta)

Número máximo de iterações atingido


(array([2., 1.]), 1001, 12753)

In [273]:
metodo_de_newton(f_x, grad_f_x, hessiana, variaveis, ponto_inicial, gama, eta)

(array([2., 1.]), 1, 1)

In [274]:
metodo_de_quase_newton(f_x, n_variaveis, grad_f_x, variaveis, ponto_inicial, gama, eta, metodo='bfgs')

[-0.01 -0.02]
[-0.00790123  0.01209877]
[ 0.00510039 -0.00699837]
[-0.01174532 -0.00474695]
[nan nan]


/var/folders/7b/0y05t52516g_yryyhc0cvctr0000gn/T/ipykernel_18367/1330231186.py:2: RuntimeWarning: invalid value encountered in scalar divide
  termo_2_1 = (1 + (((q.T @ H) @ q) / (p.T @ q)))
/var/folders/7b/0y05t52516g_yryyhc0cvctr0000gn/T/ipykernel_18367/1330231186.py:3: RuntimeWarning: invalid value encountered in scalar divide
  termo_2_2 = (p @ p.T) / (p.T @ q)
/var/folders/7b/0y05t52516g_yryyhc0cvctr0000gn/T/ipykernel_18367/1330231186.py:5: RuntimeWarning: invalid value encountered in scalar divide
  termo_3 = ((p @ (q.T @ H)) + ((H @ q) @ p.T)) / (p.T @ q)


TypeError: Invalid NaN comparison